In [1]:
# from google.colab import files

# uploaded = files.upload()

In [2]:
"""Import the required modules"""
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
def get_tables(file1, file2):
    
  # Reads in files.
  dfs = [pd.read_csv(file1), pd.read_csv(file2)]

  # Create dummies.
  a = 0
  for i, df in enumerate(dfs):
    dfs[i] = pd.get_dummies(df)
    dfs[i] = pd.concat([dfs[i].iloc[:, :4], dfs[i].iloc[:, -5+a:], dfs[i].iloc[:, 4:8+a]], axis=1)
    a += 1

  return dfs

In [10]:
file = ('welding_data_by_properties.csv', 'welding_data_by_welding_types.csv')
df = get_tables(file[0], file[1])

In [5]:
def generate_train_test(i, df):
  X = df[i].iloc[:, :9-i]
  y = df[i].iloc[:, -4:]

  return train_test_split(X, y,random_state=1, test_size=0.2)

In [19]:
for i in range(2):
    X_train, X_test, y_train, y_test = generate_train_test(i, df)

    reg = Ridge(alpha=0.1, normalize=True)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)

    print(f'\nOutputs for {file[i]}')
    print(f'\nCoefficients:')
    coefficients = reg.coef_
    print(coefficients)

    print(f'\nIntercepts:')
    intercepts = reg.intercept_
    print(intercepts)

    print(f'\nR2: {reg.score(X_test, y_test)}\n')

    print('Regression Equations:')
    col_names = [name for name in df[i].columns]

    df_col = []
    for name in df[i].columns:
        if 'property' in name:
            df_col.append(name[len('property')+1:])
        elif 'welding_type' in name:
            df_col.append(name[len('welding_type')+1:])
        else:
            df_col.append(name)

    equation = ''
    for idx, j in enumerate(coefficients):
        for idy, k in enumerate(j):
            equation += f'{round(k, 2)}*{df_col[idy]} + '
        print(f'{y_train.columns[idx]} = {round(intercepts[idx], 2)} + {equation[:-3]}')
        equation = ''


Outputs for welding_data_by_properties.csv

Coefficients:
[[   2.83711357   12.70901154   -3.36367979    8.49278845 -278.93728344
  -431.7242221  1387.49747265 -302.31442829 -126.90519944]
 [   2.42230596   11.0859444    -2.9693308     9.15282651 -225.34088724
  -343.3535095  1094.65481676 -243.27490126  -84.2041348 ]
 [   2.49878684   11.59157222   -2.97708262    8.99801773 -226.62844667
  -350.66762408 1087.91226086 -236.7758715   -74.84495495]
 [   1.39377031   10.90689045   -1.92253069    5.30224167  -79.14520433
  -299.98038831  782.4953942  -262.94817182   16.52973491]]

Intercepts:
[190.92327644 135.56600145 120.18078236 103.81323488]

R2: 0.7310102043140464

Regression Equations:
saw = 190.92 + 2.84*current + 12.71*voltage + -3.36*welding_speed + 8.49*plate_thickness + -278.94*flexural_strength + -431.72*hardness + 1387.5*heat_input + -302.31*residual_stress + -126.91*tensile_strength
smaw = 135.57 + 2.42*current + 11.09*voltage + -2.97*welding_speed + 9.15*plate_thickness + -